# Preprocessing data menggunakan TFIDF

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pickle

# Baca dataset bengkel ke dalam DataFrame
data = pd.read_csv('data_bengkel.csv')

# Menginisialisasi objek TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Mengubah teks deskripsi menjadi matriks TF-IDF
tfidf_matrix = tfidf.fit_transform(data['fitur'])

# Menyimpan model ke format .h5

In [2]:
import h5py
import numpy as np

def save_kmeans_to_h5(model, filepath):
    with h5py.File(filepath, 'w') as f:
        # Buat grup untuk model K-means
        kmeans_group = f.create_group('kmeans')

        # Simpan atribut model
        kmeans_group.attrs['model_name'] = 'K-means Model'
        kmeans_group.attrs['description'] = 'Model K-means untuk sistem rekomendasi bengkel'
        kmeans_group.attrs['n_clusters'] = model.n_clusters
        kmeans_group.attrs['random_state'] = model.random_state

        # Simpan centroid kluster
        kmeans_group.create_dataset('cluster_centers_', data=model.cluster_centers_)

        # Simpan label kluster
        kmeans_group.create_dataset('labels_', data=model.labels_)

        # Simpan jumlah iterasi yang dilakukan
        kmeans_group.attrs['n_iter_'] = model.n_iter_

# Melakukan clustering dengan K-means
kmeans = KMeans(n_clusters=15, random_state=42)
kmeans.fit(tfidf_matrix)

# Menyimpan model K-means ke dalam file .h5
save_kmeans_to_h5(kmeans, 'recommender_system.h5')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# Menampilkan hasil rekomendasi

In [3]:
# Fungsi untuk mendapatkan rekomendasi berdasarkan deskripsi bengkel
def get_recommendations(fitur, kmeans=kmeans, data=data):
    # Mengubah deskripsi menjadi vektor TF-IDF
    tfidf_vector = tfidf.transform([fitur])

    # Mengelompokkan vektor dengan model K-means
    cluster_label = kmeans.predict(tfidf_vector)[0]

    # Mendapatkan indeks bengkel dalam kluster yang sama
    recommended_indices = data[kmeans.labels_ == cluster_label].index

    # Mengembalikan nama-nama bengkel rekomendasi
    return data['nama'].iloc[recommended_indices]

# Contoh penggunaan
recommendations = get_recommendations('bengkel mekanik official')
print(recommendations)

3               bengkel mobil almet spesialis matic
5                                bengkel mobil iyan
7                 bengkel mobil kembar auto service
9             bengkel mobil kurnia jaya motor denso
14                  bengkel mobil motor anto garage
22                           bengkel mobil retrofit
27                         bengkel mobil bang sirun
31                                       istana per
40                                  bengkel pospera
140    Bengkel Mobil Mugiwara Pasar Mobil Kemayoran
Name: nama, dtype: object


# Mengevaluasi model

In [4]:
from sklearn.metrics import silhouette_score

# Menghitung inertia
inertia = kmeans.inertia_
print("Inertia:", inertia)

# Menghitung Silhouette score
silhouette = silhouette_score(tfidf_matrix, kmeans.labels_)
print("Silhouette Score:", silhouette)

Inertia: 61.25740927984211
Silhouette Score: 0.17555395951616906


Untuk mengevaluasi model sistem rekomendasi yang menggunakan K-means dapat menggunakan beberapa metrik evaluasi berikut:

1. Inertia: Inertia mengukur sejauh mana titik data dalam suatu kluster cenderung berdekatan dengan centroid kluster tersebut. Semakin rendah nilai inertia, semakin baik model K-means yang dihasilkan. Hal ini dapat menggunakan atribut inertia_ dari objek KMeans di scikit-learn untuk mendapatkan nilai inertia model.

2. Silhouette Score: Silhouette score mengukur sejauh mana objek dalam satu kluster memiliki kedekatan dengan objek dalam kluster yang sama dibandingkan dengan kluster lain. Nilai Silhouette score berkisar antara -1 hingga 1, di mana nilai yang lebih tinggi menunjukkan kualitas klustering yang lebih baik. Hal ini dapat menggunakan fungsi silhouette_score dari scikit-learn untuk menghitung nilai Silhouette score.

3. Evaluasi Subyektif: Selain metrik evaluasi numerik, penting juga untuk melakukan evaluasi subyektif terhadap rekomendasi yang dihasilkan oleh model K-means. Hal ini dapat melibatkan pengguna atau pemangku kepentingan dalam proses evaluasi untuk memperoleh umpan balik dan insight mengenai relevansi dan kualitas rekomendasi yang diberikan oleh model.